### Numeric Cores

- Take any number with four or more digits. Without changing the sequence, split the numbe into four smaller numbers. (ex. `86455` becomes `8` `6` `45` `5`)
- Next, assign each a different color, resulting in a valid mathematical equation. The first number should always be assigned teal to begin with a positive number. (ex. $+$ `8` $-$ `6` $\times$ `45` $÷$ `5`)
- If the result is a number with more than three digits, repeat the above process. The final number you obtain that is less than four digits is considered the “numeric core” of the larger number. (ex. 18)

In [1]:
import numpy as np
import operator
import pandas as pd
import sympy
import itertools
import functools
import IPython.display

In [2]:
def word_to_nums(word: str):
    return [ord(char) - 64 for char in word.strip().upper()]

word_to_nums("pigs")

[16, 9, 7, 19]

In [3]:
def nums_to_word(*nums):
    return ''.join([*(chr(n + 64) for n in nums)])

nums_to_word(16, 9, 7, 19)

'PIGS'

In [4]:
def number_splits(num, n=4):
    if type(num) is str:
        if len(num) != n:
            raise ValueError()
        yield word_to_nums(num)
        return
    digits = list(str(num))
    for digit_combo in itertools.combinations(range(len(digits)-1), n-1):
        numbers = np.split(digits, np.array(digit_combo)+1)
        yield([int(''.join([str(n) for n in number])) for number in numbers])

[*number_splits(86455)]

[[8, 6, 4, 55], [8, 6, 45, 5], [8, 64, 5, 5], [86, 4, 5, 5]]

In [5]:
[*number_splits('pigs')]

[[16, 9, 7, 19]]

In [6]:
ops = ['+', '-', '×', '÷']
op_lookup = {
    '+': operator.add,
    '-': operator.sub,
    '×': operator.mul,
    '÷': operator.truediv
}
op_orders = [ops[:1] + list(o) for o in itertools.permutations(ops[1:])]
op_orders

[['+', '-', '×', '÷'],
 ['+', '-', '÷', '×'],
 ['+', '×', '-', '÷'],
 ['+', '×', '÷', '-'],
 ['+', '÷', '-', '×'],
 ['+', '÷', '×', '-']]

In [7]:
def make_expr(syms, op_order, nums=None):
    def take_op(lhs, rhs):
        take_op.i = take_op.i+1
        return op_lookup[op_order[take_op.i]](lhs, rhs)
    take_op.i = 0
    expr = functools.reduce(take_op, syms)
    if nums:
        with sympy.evaluate(False):
            expr = expr.subs(zip(syms, nums))
    return expr

expr = make_expr(sympy.symbols('a b c d'), op_orders[0])
expr

c*(a - b)/d

In [8]:
def _possible_cores(num, include_invalid=False):
    syms = sympy.symbols('a b c d')
    if type(num) is str:
        splits = [word_to_nums(num)]
        if len(splits[0]) != 4:
            raise ValueError()
    else:
        splits = number_splits(num)
    for nums in splits:
        for op_order in op_orders:
            expr = make_expr(syms, op_order, nums)
            result = expr.doit()
            valid = result.is_integer and result > 0
            if valid or include_invalid:
                yield({
                    syms[0].name: f"{op_order[0]}{nums[0]}",
                    syms[1].name: f"{op_order[1]}{nums[1]}",
                    syms[2].name: f"{op_order[2]}{nums[2]}",
                    syms[3].name: f"{op_order[3]}{nums[3]}",
                    "expression": expr,
                    "result": result,
                    "letter": chr(result + 64)
                })

def possible_cores(num, include_invalid=False):
    return pd.DataFrame([*_possible_cores(num, include_invalid=include_invalid)])

possible_cores(86455)

,a,b,c,d,expression,result,letter
0,+8,-6,×45,÷5,45*(8 - 1*6)/5,18,R
1,+8,÷6,×45,-5,-1*5 + 8*45/6,55,w
2,+86,-4,×5,÷5,5*(86 - 1*4)/5,82,
3,+86,-4,÷5,×5,5*(86 - 1*4)/5,82,


In [79]:
def numeric_core(num, verbose=False):
    cores = possible_cores(num)
    if len(cores) == 0:
        return 0
    cores.sort_values(by="result", inplace=True)
    best = cores.iloc[0]
    if verbose:
        display(IPython.display.Math(f"{sympy.latex(best.expression)} = {best.result}"))
        # display(best.expression)
        # print(f"{num} := {best.a} {best.b} {best.c} {best.d} = {best.result}")
    result = best["result"]
    if len(str(result)) > 3:
        return numeric_core(result)
    else:
        return result

numeric_core(86455, verbose=True)

<IPython.core.display.Math object>

18

---

### Word Cores

In [ ]:
words = [
    ["pigs", "sand", "mail", "date", "head"],
    ["clam", "peak", "heat", "joya", "well"],
    ["toad", "card", "will", "tape", "legs"],
    ["tree", "road", "maid", "slab", "rock"],
    ["hand", "vase", "safe", "clay", "toes"],
]
[nums_to_word(*[numeric_core(word) for word in row]) for row in  words]

In [10]:
possible_cores('pigs')

,a,b,c,d,expression,result,letter
0,+16,-9,÷7,×19,19*(16 - 1*9)/7,19,S


In [12]:
possible_cores('heat')

,a,b,c,d,expression,result,letter
0,+8,-5,÷1,×20,20*(8 - 1*5)/1,60,|
1,+8,×5,÷1,-20,-1*20 + 8*5/1,20,T
2,+8,÷5,-1,×20,20*(-1*1 + 8/5),12,L


#### floating-point anomaly

In [13]:
numeric_core('heat', verbose=True)

<IPython.core.display.Math object>

12

In [59]:
((8.0 / 5.0) - 1.0) * 20

12.000000000000002

In [ ]:
(((+8 ×5) ÷1) -20) = 20

In [ ]:
(((+8 ÷5) -1) ×20) = 12

---
### 4-letter dictionary words grouped by their core

In [31]:
from pathlib import Path
from collections import defaultdict

In [ ]:
with Path('/usr/share/dict/words').open('r') as f:
    dict = f.read().split('\n')

core_words = defaultdict(list)

for word in dict:
    if len(word) != 4:
        continue
    core = numeric_core(word)
    if core != 0:
        core_words[core].append(word)

In [34]:
print(' '.join(core_words[20]))

blad buba calp chad clap fate fuze hade haet jati Lula malt Mayo mico naid neti ocht pail pale Pali pali palp pato pirl plea pupa rane sand Sart sart scyt sent Sikh soco Tape tape teil teli thio tidy toad Toda trod ulla vare Waac ward wase wert wint yegg yell zink zuza blad buba calp chad clap fate fuze hade haet jati Lula malt Mayo mico naid neti ocht pail pale Pali pali palp pato pirl plea pupa rane sand Sart sart scyt sent Sikh soco Tape tape teil teli thio tidy toad Toda trod ulla vare Waac ward wase wert wint yegg yell zink zuza


In [52]:
possible_cores('pail')

,a,b,c,d,expression,result,letter
0,+16,-1,÷9,×12,12*(16 - 1*1)/9,20,T
1,+16,÷1,-9,×12,12*(-1*9 + 16/1),84,
2,+16,÷1,×9,-12,-1*12 + 16*9/1,132,Ä


---

### For Comparison: floating-point version by Briggsby

In [65]:
import itertools

OPERATIONS = ['/', '-', '*']
OPERATORS = {
    '-': (lambda a, b: a - b),
    '/': (lambda a, b: a / b),
    '*': (lambda a, b: a * b),
}
ALL_OPERATOR_PERMUTATIONS = list(itertools.permutations(OPERATIONS))


def get_core(numbers: list[float]):
    if len(numbers) != 4:
        raise ValueError
    lowest_result = None
    lowest_result_number_permutation = None
    lowest_result_operator_permutation = None
    for operator_permutation in ALL_OPERATOR_PERMUTATIONS:
        result = numbers[0]
        result = OPERATORS[operator_permutation[0]](result, numbers[1])
        result = OPERATORS[operator_permutation[1]](result, numbers[2])
        result = OPERATORS[operator_permutation[2]](result, numbers[3])

        if result <= 0 or not result.is_integer():
            continue

        if lowest_result is None or result < lowest_result:
            lowest_result = result
            lowest_result_number_permutation = numbers
            lowest_result_operator_permutation = operator_permutation

    if lowest_result is None:
        return 0, None, None
    return int(lowest_result), lowest_result_number_permutation, lowest_result_operator_permutation


def get_core_from_word(word: str):
    numbers = [float(ord(char)) - 96 for char in word]
    # print(f"Solving core for {word}: {numbers}")
    core, numbers_order, operators_order = get_core(numbers)
    # print(core, numbers_order, operators_order)
    #print(chr(core + 96), end="")
    # print(core, end=",")
    return get_core(numbers)


In [64]:
get_core_from_word("heat")

t

(20, [8.0, 5.0, 1.0, 20.0], ('*', '/', '-'))

In [66]:
word_cores = []
core_words = defaultdict(list)

for word in dict:
    if len(word) != 4:
        continue
    core = numeric_core(word)
    if core != 0:
        core_words[core].append(word)
        float_core, _, _ = get_core_from_word(word.lower())
        word_cores.append({"word": word, "rational_core": core, "floating_core": float_core})

word_df = pd.DataFrame(word_cores)

In [75]:
word_df.set_index("word", inplace=True)

In [78]:
word_df[word_df.rational_core != word_df.floating_core]

,rational_core,floating_core
word,,
feat,4,10
Geat,8,15
geat,8,15
heat,12,20
liar,6,54
Onan,1,14
sect,16,0
Ural,2,36
ural,2,36
